In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import json
import time
import torch
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer

2025-03-11 18:28:01.117905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-11 18:28:01.117970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-11 18:28:01.119335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 18:28:01.128228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
ratios = [0.5, 0.6, 0.7]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
dataset_name = "multifieldqa_en"
dataset = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
model_name = 'facebook/bart-large-cnn'
model = BartForConditionalGeneration.from_pretrained(model_name)
model.to(device)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [5]:
def summarize_text(input_text, tokenizer, model, ratio):
  inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
  input_token_length = inputs['input_ids'].shape[1]  # Number of tokens in the input
  # Calculate ratio of the input token length
  target_length = int(input_token_length * ratio)
  # print(f"Input token length: {input_token_length}. Target token length (50%): {target_length}")
  summary_ids = model.generate(
    inputs['input_ids'], 
    num_beams=4,
    max_length=target_length,
    min_length=min(max(target_length - 100, 50), target_length),
    length_penalty=2.0,  # Encourage a more concise summary
    early_stopping=True
  )
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

def generate_multiple_summaries(input_text, tokenizer, model, ratio):
  idx = 0
  input_len = len(input_text)
  summary = ""
  while idx < input_len:
    # print(idx)
    end_idx = min(idx+4000, input_len)
    summary += " " + summarize_text(input_text[idx:end_idx], tokenizer, model, ratio)
    idx = end_idx + 1
  return summary

In [ ]:
start_time = time.time()
summarized_data_ratio1 = []
ctr = 0

for item in dataset:
    print(ctr)
    ctr += 1
    time1 = time.time()
    try:
        summarized_context = generate_multiple_summaries(item['context'], tokenizer, model, ratios[0])
        input_text = "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:".format(context=summarized_context, input=item["input"])
        summarized_item = item.copy()
        summarized_item["summary"] = input_text
        summarized_data_ratio1.append(summarized_item)
    except Exception as e:
        print(f"Error summarizing item: {e}")
        summarized_item = item.copy()
        summarized_item["summary"] = "Summary Failed"
        summarized_data_ratio1.append(summarized_item)
    print("time taken for this step = ", time.time() - time1)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time = ", execution_time)

output_file_path = "longbench_summarized_datasets_ratio3.json"
with open(output_file_path, "w", encoding="utf-8") as f_out:
  json.dump(summarized_data_ratio1, f_out, indent=4)

In [ ]:
start_time = time.time()
summarized_data_ratio2 = []
ctr = 0

for item in dataset:
    print(ctr)
    ctr += 1
    time1 = time.time()
    try:
        summarized_context = generate_multiple_summaries(item['context'], tokenizer, model, ratios[0])
        input_text = "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:".format(context=summarized_context, input=item["input"])
        summarized_item = item.copy()
        summarized_item["summary"] = input_text
        summarized_data_ratio2.append(summarized_item)
    except Exception as e:
        print(f"Error summarizing item: {e}")
        summarized_item = item.copy()
        summarized_item["summary"] = "Summary Failed"
        summarized_data_ratio2.append(summarized_item)
    print("time taken for this step = ", time.time() - time1)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time = ", execution_time)

output_file_path = "longbench_summarized_datasets_ratio3.json"
with open(output_file_path, "w", encoding="utf-8") as f_out:
  json.dump(summarized_data_ratio2, f_out, indent=4)

In [ ]:
start_time = time.time()
summarized_data_ratio3 = []
ctr = 0

for item in dataset:
    print(ctr)
    ctr += 1
    time1 = time.time()
    try:
        summarized_context = generate_multiple_summaries(item['context'], tokenizer, model, ratios[0])
        input_text = "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:".format(context=summarized_context, input=item["input"])
        summarized_item = item.copy()
        summarized_item["summary"] = input_text
        summarized_data_ratio3.append(summarized_item)
    except Exception as e:
        print(f"Error summarizing item: {e}")
        summarized_item = item.copy()
        summarized_item["summary"] = "Summary Failed"
        summarized_data_ratio3.append(summarized_item)
    print("time taken for this step = ", time.time() - time1)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time = ", execution_time)

output_file_path = "longbench_summarized_datasets_ratio3.json"
with open(output_file_path, "w", encoding="utf-8") as f_out:
  json.dump(summarized_data_ratio3, f_out, indent=4)

0
time taken for this step =  11.741273164749146
1
time taken for this step =  96.67515969276428
2
time taken for this step =  72.86250400543213
3
time taken for this step =  103.09430313110352
4
time taken for this step =  63.33922576904297
5
time taken for this step =  86.64067077636719
6
time taken for this step =  66.14185166358948
7
time taken for this step =  22.555176973342896
8
time taken for this step =  10.58476972579956
9
time taken for this step =  100.92014408111572
10
time taken for this step =  81.70458030700684
11
time taken for this step =  84.03439712524414
12
time taken for this step =  99.49449801445007
13
time taken for this step =  63.003211975097656
14
time taken for this step =  77.21896409988403
15
time taken for this step =  20.822147130966187
16
time taken for this step =  70.05341172218323
17
time taken for this step =  77.20126080513
18
time taken for this step =  81.54230165481567
19
time taken for this step =  30.475788593292236
20
time taken for this ste